# Dependents

In [ ]:
%pylab inline
from scipy import optimize
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import os
from MP.mat import voigt
ijv=voigt.ijv
vij=voigt.vij

p_home = os.getcwd()
p_load = '/Users/yj/repo/vpsc/vpsc-dev-yld/src/py_pack/yld_hah/'

os.chdir(p_load)
import lib_dat
os.chdir(p_home)

In [ ]:
def ys_temp(ax):
    ax.grid()
    ax.set_aspect('equal')
    ax.set_xlabel(r'$\Sigma_\mathrm{11}$',fontsize=17)
    ax.set_ylabel(r'$\Sigma_\mathrm{22}$',fontsize=17)
    
def pi_proj(sd):
    """
    Deviatoric stress to pi-plane projection
    """
    sq6 = np.sqrt(6.)
    sq2 = np.sqrt(2.)    
    x = 2.*sd[0]/sq6 - sd[1] / sq6 - sd[2] / sq6
    y =                sd[1] / sq2 - sd[2] / sq2
    return x,y 

def devit(x,p=0.):
    """
    Convert it to a deviatoric space with
    hydrostatic stress of <p>
    
    Argument
    --------
    x  : Conver stress <x> to a deviatric
    p  : pressure (optional, if non-zero,
                   translate the stress along hydrostatic axis)
    """
    x=np.array(x,dtype='float')
    m=x[:3].sum()/3.
    x[:3]=x[:3]-m
    
    if p!=0:
        x[:3]=x[:3]+p/3.    
    return x

## Quadratic Hill Yield criterion

$\phi(\sigma)=\langle{\sigma_1^2+\frac{R_0(1+R_90)}{R_90(1+R_0)}\sigma_2^2 - \frac{2R_0}{1+R_0}\sigma_1\sigma_2}\rangle^{1/2.}$

In [ ]:
ijv.shape
print ijv

In [ ]:
def quadratic_hill(sigma,R0,R90):
    """
    Sigma should be principal stress
    """
    #Assuming sigma is 6d

    mat33=np.zeros((3,3))
    for k in xrange(6):
        i,j = ijv[:,k]
        mat33[i,j] = sigma[k]
        if i!=j: mat33[j,i]=sigma[k]

    w,v = numpy.linalg.eig(mat33)
    s1,s2,s3 = w
    
    
    phi = s1**2 + R0 * (1+R90) / (R90*(1+R0))*s2**2 - \
         (2*R0)/(1+R0)*s1*s2
    
    return phi**(1./2.)

In [ ]:
def th_2_planestress(th):
    """
    Given theta, return the stress.

    Argument
    --------
    th
    """
    sigma=np.zeros(6)
    sigma[0]=np.cos(th)
    sigma[1]=np.sin(th)    
    return sigma

def th_planestress(th=0.,**kwargs):
    """
    
    Return stress tensors that gives the same
    size (value) of phi
    
    Argument
    --------
    th
    a
    """
    Sigma = th_2_planestress(th)
    y     = quadratic_hill(Sigma,**kwargs)
    return Sigma / y

## Hill yield loci

In [ ]:
def hill_quad_ys(nths,**kwargs):
    ths=np.linspace(-pi,pi,nths)
    locus_ps=np.zeros((2,nths))

    xy=[]
    for i in xrange(len(ths)):
        ys = th_planestress(ths[i],**kwargs) ## stress tensors
        locus_ps[:,i]=ys[0],ys[1]
        sd = np.zeros(6)
        sd[0],sd[1] = ys[0],ys[1]
        sd = devit(sd)
        x, y = pi_proj(sd)
        xy.append([x,y])

    locus_pi=np.array(xy).T
    return locus_ps, locus_pi

locus_ps,locus_pi = hill_quad_ys(nths=1000,R0=1.,R90=2.)

fig=plt.figure(figsize=(7,3))
ax1=fig.add_subplot(121); ax2=fig.add_subplot(122)
ax1.plot(locus_ps[0],locus_ps[1],'k-')
ax2.plot(locus_pi[0],locus_pi[1],'k-')

## deco
lib_dat.pi_rad(ax2,rot=150,rel=1.5)
ys_temp(ax1)

## Associated Flow Rule

- Tensorial representation of associated flow rule

$\dot{\varepsilon}_{ij} = \lambda \frac{\partial \phi(\mathbf{\sigma})}{\partial \sigma_{ij}}$

- 6D vectorial representation

$\dot{\varepsilon}_k=\lambda \frac{\partial \phi(\mathrm{\sigma})}{\partial \sigma_k}$




- Numerical approximation of the above is:

$\dot{\varepsilon}_k=\lambda\ \delta_{ki} \frac{\phi(\sigma_1,\sigma_2,...,\sigma+\Delta\sigma_i) - \phi(\sigma_1,\sigma_2,...,\sigma_i+\Delta\sigma)}{2\Delta \sigma} $

With infinitesimally small $\Delta \sigma$

$\Delta \sigma$ should be sufficiently small.

In [ ]:
def assoc_flow(s6=[1,0,0,0,0,0],lamb=1.,**kwargs):
    """
    Argument
    --------
    s6  (6D cauchy stress)
    """
    dlt = 1e-10
    phi = quadratic_hill(s6,**kwargs)
    s1  = np.zeros(6); s2  = np.zeros(6)
    dki = np.identity(6)
    e_k = np.zeros(6)

    for k in xrange(6):
        dum=0.
        s1=np.zeros(6);
        s2=np.zeros(6);
        for i in xrange(6):
            s1[i] = s[i] + dki[k,i] * dlt
            s2[i] = s[i] - dki[k,i] * dlt
        e_k[k] = lamb*(quadratic_hill(s1,**kwargs) - quadratic_hill(s2,**kwargs))/(2*dlt)
        
    e_k[2]=-e_k[:2].sum()
    return e_k

## Test Associated Flow Rule

In [ ]:
R0=3.
R90=0.5
locus_ps,locus_pi = hill_quad_ys(nths=1000,R0=R0,R90=R90)

fig=plt.figure(figsize=(3.5,3))
ax1=fig.add_subplot(111)

nstp=15
ax1.plot(locus_ps[0],locus_ps[1],'k-')
for i in xrange(len(locus_ps[0])/nstp):
    x,y = locus_ps[0][i*nstp],locus_ps[1][i*nstp]
    th  = np.arctan2(y,x)
    s   = np.array([x,y,0,0,0,0])

    de = assoc_flow(s,0.02,R0=R0,R90=R90)
    #de[:2]=de[:2]/np.sqrt((de[:2]**2).sum())
    TH  = np.arctan2(de[1],de[0])
    r = 0.15
    dx,dy = r*cos(TH),r*sin(TH)
    ax1.arrow(x,y,dx,dy,color='red')
    # ax1.plot(x,y,'k.')

ys_temp(ax1)